# Benchmark SAC with Original Implementation

Github Repo: git@github.com:XiaohanZhangCMU/sderl.git. 

Note: sderl/sac works only with continuous action environments



# Work node preparation

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import os
os.getcwd()
os.chdir('/content/gdrive/My Drive/ActiveProjects/sderl/')

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# Stable Baselines only supports tensorflow 1.x for now
# %tensorflow_version 1.x
# !apt install swig cmake libopenmpi-dev zlib1g-dev
# !pip install stable-baselines[mpi]==2.10.0 box2d box2d-kengz

In [ ]:
# !apt install swig cmake libopenmpi-dev zlib1g-dev
!pip install -e . 
# !pip install box2d box2d-kengz

Obtaining file:///content/gdrive/My%20Drive/ActiveProjects/sderl
     |████████████████████████████████| 1.4MB 14.3MB/s 
     |████████████████████████████████| 450kB 43.1MB/s 
  Created wheel for mpi4py: filename=mpi4py-3.0.3-cp36-cp36m-linux_x86_64.whl size=2074489 sha256=3b752d61f301c57d9d00afded779fbdc06c9230b2b7a4075e90432292ff4cf0d
  Stored in directory: /root/.cache/pip/wheels/18/e0/86/2b713dd512199096012ceca61429e12b960888de59818871d6
Successfully built mpi4py
  Running setup.py develop for sderl


#Run experiment from script (GPU) (LunarLanderContinuous-v2)

In [ ]:
%%time
from sderl import sac_pytorch as sac
import torch
import gym

env_fn = lambda : gym.make('LunarLanderContinuous-v2')

ac_kwargs = dict(hidden_sizes=[64,64], activation=torch.nn.ReLU)

logger_kwargs = dict(output_dir='./output_dir_tmp', exp_name='sac_lunarcontinuous-v2')

sac(env_fn=env_fn, ac_kwargs=ac_kwargs, steps_per_epoch=5000, epochs=250, logger_kwargs=logger_kwargs)

Logging data to ./output_dir_tmp/progress.txt
Saving config:

{
    "ac_kwargs":	{
        "activation":	"ReLU",
        "hidden_sizes":	[
            64,
            64
        ]
    },
    "actor_critic":	"MLPActorCritic",
    "alpha":	0.2,
    "batch_size":	100,
    "device":	"cuda",
    "env_fn":	"<function <lambda> at 0x7f933001f158>",
    "epochs":	250,
    "exp_name":	"sac_lunarcontinuous-v2",
    "gamma":	0.99,
    "logger":	{
        "<sderl.utils.logx.EpochLogger object at 0x7f9317645a90>":	{
            "epoch_dict":	{},
            "exp_name":	"sac_lunarcontinuous-v2",
            "first_row":	true,
            "log_current_row":	{},
            "log_headers":	[],
            "output_dir":	"./output_dir_tmp",
            "output_file":	{
                "<_io.TextIOWrapper name='./output_dir_tmp/progress.txt' mode='w' encoding='UTF-8'>":	{
                    "mode":	"w"
                }
            }
        }
    },
    "logger_kwargs":	{
        "exp_name":	"sac_lunarco

# GPU Profiling

In [ ]:
!python -m torch.utils.bottleneck sderl/algos/pytorch/sac/sac.py --env LunarLanderContinuous-v2 --epochs 5 --update_after=0

`bottleneck` is a tool that can be used as an initial step for debugging
bottlenecks in your program.

It summarizes runs of your script with the Python profiler and PyTorch's
autograd profiler. Because your script will be profiled, please ensure that it
exits in a finite amount of time.

For more complicated uses of the profilers, please see
https://docs.python.org/3/library/profile.html and
https://pytorch.org/docs/master/autograd.html#profiler for more information.
Running environment analysis...
Running your script with cProfile
Logging data to /content/gdrive/My Drive/ActiveProjects/sderl/data/sac/sac_s0/progress.txt
Saving config:

{
    "ac_kwargs":	{
        "hidden_sizes":	[
            64,
            64
        ]
    },
    "actor_critic":	"MLPActorCritic",
    "alpha":	0.2,
    "batch_size":	256,
    "device":	"cuda",
    "env_fn":	"<function <lambda> at 0x7f4049144510>",
    "epochs":	5,
    "exp_name":	"sac",
    "gamma":	0.99,
    "logger":	{
        "<sderl.utils.logx.E

In [ ]:
%%time
# !pip install ray[rllib]  # also recommended: ray[debug]
!rllib train --run=SAC --env=LunarLanderContinuous-v2 --config='{"num_gpus": 1}'

Streaming output truncated to the last 5000 lines.
+------------------------------------------+----------+-----------------+--------+------------------+---------+----------+
| Trial name                               | status   | loc             |   iter |   total time (s) |      ts |   reward |
|------------------------------------------+----------+-----------------+--------+------------------+---------+----------|
| SAC_LunarLanderContinuous-v2_115f7_00000 | RUNNING  | 172.28.0.2:2047 |  15081 |          25555.2 | 1509500 |  282.722 |
+------------------------------------------+----------+-----------------+--------+------------------+---------+----------+


== Status ==
Memory usage on this node: 3.2/12.7 GiB
Using FIFO scheduling algorithm.
Resources requested: 1/2 CPUs, 1/1 GPUs, 0.0/7.37 GiB heap, 0.0/2.54 GiB objects
Result logdir: /root/ray_results/default
Number of trials: 1 (1 RUNNING)
+------------------------------------------+----------+-----------------+--------+----------

In [ ]:
!ls ~/

In [ ]:
import torch
torch.cuda.device_count()

1

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Jul 17 16:53:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |     11MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.7 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.
